In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
from monty.serialization import loadfn, dumpfn

In [16]:
from mp_api.client import MPRester
mpr = MPRester()

In [75]:
from ase.filters import FrechetCellFilter
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.optimize import LBFGS
from pymatgen.io.ase import AseAtomsAdaptor

predictor = pretrained_mlip.get_predict_unit("uma-m-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="omat") # use "omat" - inorganic materials

checkpoints/uma-m-1p1.pt:   0%|          | 0.00/11.2G [00:00<?, ?B/s]

In [76]:
def relax_structure(structure): # takes in pymatgen structure
    atoms = structure.to_ase_atoms()
    atoms.calc = calc

    opt = LBFGS(atoms) # Delete this FrechetCellFilter when doing the defect structures
    opt.run(0.01, 1000)

    structure1 = AseAtomsAdaptor.get_structure(atoms)
    energy = atoms.get_potential_energy()

    return structure1, energy 

In [19]:
from doped.chemical_potentials import CompetingPhases
from pymatgen.entries.computed_entries import ComputedEntry
from doped.chemical_potentials import CompetingPhasesAnalyzer
from pymatgen.analysis.chempot_diagram import ChemicalPotentialDiagram

In [77]:
calculated_entries = {}

def calc_competing_phase(comp): # Pass in formula of species
    cp = CompetingPhases(composition=comp, extrinsic='Cr', energy_above_hull=0.05)
    entries = []
    for entry in cp.entries:
        key =  entry.data['material_id'] + entry.structure.composition.reduced_formula
        
        if key in calculated_entries:
            new_entry = calculated_entries[key]
        else:
            relaxed_structure, energy = relax_structure(entry.structure)
            new_entry = ComputedEntry(
                relaxed_structure.composition,
                energy, 
                data = {'material_id':entry.data['material_id']}
            )
            calculated_entries[key] = new_entry 
        entries.append(new_entry)
        
    return cp, entries

In [46]:
def calc_competing_phase(comp): # Pass in formula of species
    cp = CompetingPhases(composition=comp, extrinsic='Cr', energy_above_hull=0.05)
    print(cp)

    entries = []

    print(cp.entries)
    for entry in cp.entries:
        print(entry.composition)
        relaxed_structure, energy = relax_structure(entry.structure)
        new_entry = ComputedEntry(
            relaxed_structure.composition,
            energy, 
            data = {'material_id':entry.data['material_id']}
        )
    # print(new_entry)
        entries.append(new_entry)

    return cp, entries

In [78]:
def calc_competing_phase_analysis(comp, entries):
    
    cpa = CompetingPhasesAnalyzer(composition=comp, entries=entries)
    
    return cpa.chempots

In [48]:
relaxed_defect = loadfn('../data/doped_defect_energies.json')

In [ ]:
for key, val in relaxed_defect.items():
    val = relaxed_defect[key]
    formula = val['formula']
    cp, entries = calc_competing_phase(formula)
    chem_pot = calc_competing_phase_analysis(formula, entries)
    if 'competing phases' not in relaxed_defect[key]:
        relaxed_defect[key]['competing phases'] = {}
    relaxed_defect[key]['competing phases']['chemical potentials'] = chem_pot
    relaxed_defect[key]['competing phases']['entries'] = entries

/home/atshaam/miniconda3/envs/project_env/lib/python3.11/site-packages/doped/chemical_potentials.py:765: UserWarning:

Duplicate entry names found for generated competing phases: AlPO4_Pbca_EaH_0.012!

/home/atshaam/miniconda3/envs/project_env/lib/python3.11/site-packages/doped/chemical_potentials.py:765: UserWarning:

Duplicate entry names found for generated competing phases: AlPO4_Pbca_EaH_0.012!



       Step     Time          Energy          fmax
LBFGS:    0 11:41:23     -381.048998        0.033537
LBFGS:    1 11:41:23     -381.049326        0.032588
LBFGS:    2 11:41:24     -381.050169        0.051802
LBFGS:    3 11:41:24     -381.050978        0.056065
LBFGS:    4 11:41:25     -381.051417        0.032707
LBFGS:    5 11:41:25     -381.051611        0.025782
LBFGS:    6 11:41:26     -381.051768        0.018328
LBFGS:    7 11:41:26     -381.051951        0.024989
LBFGS:    8 11:41:27     -381.052126        0.024364
LBFGS:    9 11:41:27     -381.052260        0.021435
LBFGS:   10 11:41:28     -381.052390        0.017935
LBFGS:   11 11:41:28     -381.052504        0.019482
LBFGS:   12 11:41:29     -381.052637        0.015394
LBFGS:   13 11:41:29     -381.052733        0.012416
LBFGS:   14 11:41:30     -381.052798        0.010108
LBFGS:   15 11:41:30     -381.052843        0.009577
       Step     Time          Energy          fmax
LBFGS:    0 11:41:30      -19.058099        0.0000

/home/atshaam/miniconda3/envs/project_env/lib/python3.11/site-packages/doped/chemical_potentials.py:765: UserWarning:

Duplicate entry names found for generated competing phases: P2O5_NA_EaH_0!



       Step     Time          Energy          fmax
LBFGS:    0 11:59:52     -106.196604        0.053932
LBFGS:    1 11:59:53     -106.196902        0.043495
LBFGS:    2 11:59:53     -106.197472        0.006747
       Step     Time          Energy          fmax
LBFGS:    0 11:59:54       -5.716220        0.008224
       Step     Time          Energy          fmax
LBFGS:    0 11:59:54      -16.060173        0.041232
LBFGS:    1 11:59:54      -16.060231        0.038140
LBFGS:    2 11:59:55      -16.060626        0.009198
       Step     Time          Energy          fmax
LBFGS:    0 11:59:55      -14.110009        0.027185
LBFGS:    1 11:59:55      -14.110041        0.025390
LBFGS:    2 11:59:56      -14.110279        0.002306
       Step     Time          Energy          fmax
LBFGS:    0 11:59:56      -12.002485        0.000001
       Step     Time          Energy          fmax
LBFGS:    0 11:59:57      -13.507328        0.000000
       Step     Time          Energy          fmax
LBFGS: 

/home/atshaam/miniconda3/envs/project_env/lib/python3.11/site-packages/doped/chemical_potentials.py:765: UserWarning:

Duplicate entry names found for generated competing phases: CrCl3_NA_EaH_0.002!



...

       Step     Time          Energy          fmax
LBFGS:    0 12:12:40     -311.060544        0.015490
LBFGS:    1 12:12:40     -311.060590        0.010995
LBFGS:    2 12:12:41     -311.060624        0.007418
       Step     Time          Energy          fmax
LBFGS:    0 12:12:41      -20.897692        0.010363
LBFGS:    1 12:12:42      -20.897704        0.010320
LBFGS:    2 12:12:42      -20.901758        0.009857
       Step     Time          Energy          fmax
LBFGS:    0 12:12:42     -167.698384        0.096776
LBFGS:    1 12:12:43     -167.699711        0.094963
LBFGS:    2 12:12:43     -167.705791        0.077562
LBFGS:    3 12:12:44     -167.706977        0.072563
LBFGS:    4 12:12:44     -167.713199        0.100660
LBFGS:    5 12:12:44     -167.716020        0.099316
LBFGS:    6 12:12:45     -167.719055        0.091184
LBFGS:    7 12:12:45     -167.721976        0.106174
LBFGS:    8 12:12:46     -167.725834        0.088890
LBFGS:    9 12:12:46     -167.729101        0.086897

/home/atshaam/miniconda3/envs/project_env/lib/python3.11/site-packages/doped/chemical_potentials.py:765: UserWarning:

Duplicate entry names found for generated competing phases: AlPO4_NA_EaH_0.0096!



       Step     Time          Energy          fmax
LBFGS:    0 12:18:01     -426.244263        0.031240
LBFGS:    1 12:18:02     -426.244542        0.021673
LBFGS:    2 12:18:02     -426.244774        0.021013
LBFGS:    3 12:18:03     -426.245133        0.033162
LBFGS:    4 12:18:03     -426.245659        0.038160
LBFGS:    5 12:18:04     -426.246231        0.029115
LBFGS:    6 12:18:04     -426.246663        0.034734
LBFGS:    7 12:18:05     -426.246869        0.025931
LBFGS:    8 12:18:06     -426.246975        0.014835
LBFGS:    9 12:18:06     -426.247086        0.017773
LBFGS:   10 12:18:07     -426.247277        0.024323
LBFGS:   11 12:18:07     -426.247452        0.030992
LBFGS:   12 12:18:08     -426.247593        0.025921
LBFGS:   13 12:18:08     -426.247681        0.013321
LBFGS:   14 12:18:09     -426.247727        0.009192
       Step     Time          Energy          fmax
LBFGS:    0 12:18:09      -24.273147        0.133570
LBFGS:    1 12:18:10      -24.276545        0.1326

/home/atshaam/miniconda3/envs/project_env/lib/python3.11/site-packages/doped/chemical_potentials.py:765: UserWarning:

Duplicate entry names found for generated competing phases: Al2O3_NA_EaH_0.0319!

/home/atshaam/miniconda3/envs/project_env/lib/python3.11/site-packages/doped/chemical_potentials.py:765: UserWarning:

Duplicate entry names found for generated competing phases: Al(HO)3_P-1_EaH_0.0319!

/home/atshaam/miniconda3/envs/project_env/lib/python3.11/site-packages/doped/chemical_potentials.py:765: UserWarning:

Duplicate entry names found for generated competing phases: Al(HO)3_P-1_EaH_0.0319!



       Step     Time          Energy          fmax
LBFGS:    0 12:21:41     -212.476865        0.030173
LBFGS:    1 12:21:41     -212.477088        0.029637
LBFGS:    2 12:21:42     -212.477424        0.028450
LBFGS:    3 12:21:43     -212.477998        0.040312
LBFGS:    4 12:21:43     -212.478448        0.026098
LBFGS:    5 12:21:44     -212.478618        0.016421
LBFGS:    6 12:21:44     -212.478694        0.010989
LBFGS:    7 12:21:45     -212.478755        0.012902
LBFGS:    8 12:21:45     -212.478803        0.011372
LBFGS:    9 12:21:46     -212.478847        0.011030
LBFGS:   10 12:21:46     -212.478906        0.011810
LBFGS:   11 12:21:47     -212.479019        0.024029
LBFGS:   12 12:21:47     -212.479200        0.033405
LBFGS:   13 12:21:48     -212.479410        0.029862
LBFGS:   14 12:21:49     -212.479541        0.014091
LBFGS:   15 12:21:49     -212.479610        0.009897
       Step     Time          Energy          fmax
LBFGS:    0 12:21:50       -6.469874        0.3310

In [74]:
key = 'mp-27863'
val = relaxed_defect[key]
formula = val['formula']
cp, entries = calc_competing_phase(formula)
chem_pot = calc_competing_phase_analysis(formula, entries)
if 'competing phases' not in relaxed_defect[key]:
    relaxed_defect[key]['competing phases'] = {}
relaxed_defect[key]['competing phases']['chemical potentials'] = chem_pot
relaxed_defect[key]['competing phases']['entries'] = entries

       Step     Time          Energy          fmax
LBFGS:    0 11:34:39      -35.212154        0.044419
LBFGS:    1 11:34:39      -35.212215        0.037003
LBFGS:    2 11:34:39      -35.212352        0.014507
LBFGS:    3 11:34:39      -35.212359        0.010158
LBFGS:    4 11:34:40      -35.212363        0.001252
       Step     Time          Energy          fmax
LBFGS:    0 11:34:40      -19.027752        0.000000
       Step     Time          Energy          fmax
LBFGS:    0 11:34:40       -3.742247        0.000000
       Step     Time          Energy          fmax
LBFGS:    0 11:34:40       -8.801036        1.091409
LBFGS:    1 11:34:41       -8.788137        1.800234
LBFGS:    2 11:34:41       -8.806875        0.083152
LBFGS:    3 11:34:41       -8.806909        0.007148
       Step     Time          Energy          fmax
LBFGS:    0 11:34:41       -3.467600        0.401731
LBFGS:    1 11:34:41       -3.470439        0.091459
LBFGS:    2 11:34:42       -3.470590        0.001769
   

/home/atshaam/miniconda3/envs/project_env/lib/python3.11/site-packages/doped/chemical_potentials.py:765: UserWarning:

Duplicate entry names found for generated competing phases: Al2O3_NA_EaH_0.0302!



In [ ]:
calculated_entries.items()

In [35]:
for key in relaxed_defect.keys():
     del relaxed_defect[key]['competing phases']['cp data']

In [36]:
dumpfn(relaxed_defect, '../data/competing_phase_calcs2.json')

In [72]:
entries[0].composition.reduced_formula

'AlClO'